In [28]:
!pip install openai==0.28.0
from google.colab import files
import openai
import time
import pandas as pd
import json
import sys


In [29]:
# -------------------------------------------------------------
# (1) 설정값
# -------------------------------------------------------------
API_KEY_FILE = "/content/가영 API for 논문.txt"  # 예) 코랩에 업로드한 파일
INPUT_XLSX = "/content/6. 700개_news media.xlsx"        # 기사 데이터가 들어있는 엑셀 파일
OUTPUT_EXCEL = "6. 700개_output_actor_statement.xlsx"  # 최종 결과를 저장할 엑셀 파일

In [30]:
# -------------------------------------------------------------
# (2) GPT 프롬프트 템플릿 (JSON 출력용, 중괄호 이스케이프 적용)
# -------------------------------------------------------------
PROMPT_TEMPLATE = """\
당신은 하나의 기사(텍스트)에서 여러 행위자(Actor)의
“명시적·암시적 발언(혹은 의견, 입장, 계획)”만 추출하여
**JSON 배열** 형태로 정리하는 업무를 수행합니다.

[기사 데이터]
- ID: {id_val}
- year: {year_val}
- keyword: {keyword_val}
- link: {link_val}
- source: {source_val}

아래 지침을 철저히 지키세요:

1) 발언(Statement)이 “직접 인용”이든, “간접 화법”이든
   해당 행위자가 **무언가를 말했거나 의견·입장을 표명**한 경우에만
   하나의 객체(object)를 생성합니다.
   - 단순히 기관·기업·제품 등이 언급된 것만으로는 무시합니다.
   - 만약 발언이 전혀 없으면 => **빈 JSON 배열([])**만 출력하고 끝냅니다.
   - “발언이 없으므로 생성 불가” 같은 안내 문구는 절대 출력하지 말 것.

2) 행위자(Actor) 정보
   - actor_name: 인명이 있으면 그대로, 없으면 "N/A"
   - actor_affiliation: 소속(기업/기관명). 없으면 "N/A"
   - actor_position: 직함(사장, 장관, 연구원 등). 없으면 "N/A"
   - 예) “포럼 관계자는 …라고 말했다” →
    actor_name="N/A", actor_affiliation="포럼", actor_position="관계자"

3) Statement(발언)
   - 기사 원문에서 해당 인물이 말하거나 표명한 문장을 **그대로** 넣어주세요.
   - 길면 여러 문장이라도 모두 포함(토큰 제한상 잘리지 않을 범위 내에서).
   - 한 actor가 기사에서 여러 번 발언했다면, **하나의 statement에 모두 연결** (예: 문장 사이에 "//" 등).
     → **절대 동일 actor를 여러 행으로 나누지 마세요.**
   - 기사 문맥상 명백하지 않은 추정은 제외하세요.

4) **간접 표현 포함 지침 (중요)**
   - 기사 본문에서 명시적인 인용구(“~~라고 말했다”)가 없더라도,
     특정 행위자의 의견·평가가 **분명히** 언급된 경우(예: “행위자 A가 ~를 어렵다고 판단했다”,
     “A와 B가 ~라고 지적했다” 등), statement에 포함하세요.
   - 다만, **기자가 임의로 추정**하는 등 그 주체의 명백한 입장인지 불분명한 서술(“~한 것으로 보인다”)은 제외하세요.
   - 실제 인용문이 아닌 간접 표현임을 표시하려면, "[간접 지적]: ~~~" 형태로 앞에 태그를 달아 주세요.
     예) statement="[간접 지적]: 목표 달성이 어렵다고 밝혔다"

5) 기타 주의
   - actor_position에 statement나 다른 정보가 들어가지 않도록 주의.
   - **‘발언이 없으므로 CSV 행을 생성할 수 없음’ 같은 문장은 출력 금지**.
   - **동일 기사 내** 여러 actor가 발언했다면, 각각의 actor마다 행을 만들되,
     **동일 actor**라면 statement를 합쳐 한 행만 생성.

6) **도메인 관련 지침(중요)**
   - 본 작업은 "수소 경제", "수소 에너지", "연료 전지" 등 '수소' 관련 주제에 대한 기사입니다.
   - 기사에 언급되더라도, **수소 분야와 전혀 관련 없는 발언**이라면 무시하고 추출하지 마세요.
     예: 개인 결혼, 개인적 신상 이야기 등 수소와 무관한 내용은 **배제**.

7) 출력 포맷(JSON 배열)
   - 전체 결과는 **JSON 배열** 형태.
   - 예시(중괄호 이스케이프 주의):
     [
       {{
         "ID": "{id_val}",
         "year": "{year_val}",
         "keyword": "{keyword_val}",
         "text": "기사 전체 본문 그대로 or 일부",
         "link": "{link_val}",
         "source": "{source_val}",
         "actor_name": "예시",
         "actor_affiliation": "예시",
         "actor_position": "예시",
         "statement": "예시 발언"
       }},
       ...
     ]
   - 발언이 없으면 빈 배열([])만 출력.
   - 배열 외에 다른 텍스트나 설명은 출력하지 말 것.
   - text 필드에 기사 원문을 그대로 넣어도 되고, 길면 그대로 넣으세요.

이제 아래 text(기사 본문)를 분석해 주세요:

\"\"\"{text_val}\"\"\"
"""